In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats

# Data
data = {
    "Study": ["Reece (2007)", "Rooban et al. (2008)", "Morio et al. (2008)", "Gupta et al. (2012)", "Nives Protrka et al. (2013)", "Rommel et al. (2016)", "Shetty et al. (2016)", 
              "Aguilar-Zinser et al. (2008)", "Badel et al. (2014)", "Tanner et al. (2014)", "Tanner et al. (2015)", "Sharma et al. (2018)"],
    "OR": [1.97, 1.69, 7.13, 1.92, 24.10, 4.80, 4.06, 
           1.04, 1.04, 1.57, 1.85, 1.67],
    "Lower CI": [1.10, 1.01, 3.67, 1.21, 13.18, 2.80, 2.22, 
                 0.73, 0.64, 1.66, 2.05, 0.34],
    "Upper CI": [3.54, 2.83, 13.82, 3.04, 44.08, 8.20, 7.44, 
                 1.23, 1.04, 2.10, 3.15, 1.16],
    "Sample Size": [233, 100, 18, 126, 200, 200, 571, 
                    629, 505, 612, 630, 200]
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Calculate LogOR and Standard Error (SE)
df["LogOR"] = np.log(df["OR"])
df["SE"] = (np.log(df["Upper CI"]) - np.log(df["Lower CI"])) / 3.92

# Adjust weights using sample size
df["Weight"] = df["Sample Size"] / (df["SE"] ** 2)

# Calculate the combined effect (fixed-effect model)
combined_logOR = np.sum(df["LogOR"] * df["Weight"]) / np.sum(df["Weight"])
combined_SE = np.sqrt(1 / np.sum(df["Weight"]))  # SE of the combined effect

# Calculate 95% CI for the combined effect
z_score = stats.norm.ppf(0.975)  # 95% confidence level
lower_CI_combined = np.exp(combined_logOR - z_score * combined_SE)
upper_CI_combined = np.exp(combined_logOR + z_score * combined_SE)

# Calculate I²
Q = np.sum(df["Weight"] * (df["LogOR"] - combined_logOR) ** 2)  # Cochran's Q statistic
df_ = len(df) - 1  # Degrees of freedom
I2 = 100 * (Q - df_) / Q if Q > df_ else 0  # Calculate I²

# Output results
print("Fixed-effect meta-analysis result:")
print(f"Combined OR: {np.exp(combined_logOR):.2f} [{lower_CI_combined:.2f}; {upper_CI_combined:.2f}]")
print(f"I²: {I2:.2f}%")

# Calculate Q-statistic (Cochran's Q) and degrees of freedom
Q = np.sum(df["Weight"] * (df["LogOR"] - combined_logOR) ** 2)
df_ = len(df) - 1  # Degrees of freedom

# Estimate between-study variance (tau^2) using DerSimonian-Laird method
tau2 = max((Q - df_) / np.sum(df["Weight"]), 0)

# Adjust weights for the random-effects model
df["Weight_random"] = 1 / (df["SE"]**2 + tau2)

# Calculate the new pooled estimate using random-effects weights
combined_logOR_random = np.sum(df["LogOR"] * df["Weight_random"]) / np.sum(df["Weight_random"])
combined_SE_random = np.sqrt(1 / np.sum(df["Weight_random"]))  # SE of the combined effect for random-effects model

# Calculate 95% CI for the combined effect using random-effects model
lower_CI_combined_random = np.exp(combined_logOR_random - 1.96 * combined_SE_random)
upper_CI_combined_random = np.exp(combined_logOR_random + 1.96 * combined_SE_random)

# Calculate I² for random-effects model
I2_random = 100 * (Q - df_) / Q if Q > df_ else 0

# Output results for random-effects model
print("\nRandom-effects meta-analysis result:")
print(f"Combined OR: {np.exp(combined_logOR_random):.2f} [{lower_CI_combined_random:.2f}; {upper_CI_combined_random:.2f}]")
print(f"I²: {I2_random:.2f}%")

# Initialize lists to store results
combined_ORs = []
I2_values = []

# Sensitivity Analysis: Exclude each study one by one
print("\nSensitivity Analysis:")
for index, study in df.iterrows():
    df_sensitivity = df.drop(index)  # Remove one study at a time
    
    # Calculate LogOR and SE for the remaining studies
    logOR_sensitivity = np.log(df_sensitivity["OR"])
    SE_sensitivity = (np.log(df_sensitivity["Upper CI"]) - np.log(df_sensitivity["Lower CI"])) / 3.92
    weights_sensitivity = df_sensitivity["Sample Size"] / (SE_sensitivity ** 2)

    # Random effects meta-analysis for the remaining studies
    combined_logOR_sensitivity = np.sum(logOR_sensitivity * weights_sensitivity) / np.sum(weights_sensitivity)
    combined_SE_sensitivity = np.sqrt(1 / np.sum(weights_sensitivity))  # SE for the combined effect
    Q_sensitivity = np.sum(weights_sensitivity * (logOR_sensitivity - combined_logOR_sensitivity) ** 2)
    df_ = len(df_sensitivity) - 1  # Degrees of freedom for the remaining studies
    tau2_sensitivity = max((Q_sensitivity - df_) / np.sum(weights_sensitivity), 0)
    
    # Recalculate weights for random effects
    df_sensitivity["Weight_random"] = 1 / (SE_sensitivity**2 + tau2_sensitivity)
    
    # Combined OR for random effects
    combined_logOR_random_sensitivity = np.sum(logOR_sensitivity * df_sensitivity["Weight_random"]) / np.sum(df_sensitivity["Weight_random"])
    combined_SE_random_sensitivity = np.sqrt(1 / np.sum(df_sensitivity["Weight_random"]))  # SE for random effects
    
    # Confidence intervals for combined OR
    lower_CI_combined_random_sensitivity = np.exp(combined_logOR_random_sensitivity - 1.96 * combined_SE_random_sensitivity)
    upper_CI_combined_random_sensitivity = np.exp(combined_logOR_random_sensitivity + 1.96 * combined_SE_random_sensitivity)
    
    # Calculate I²
    I2_sensitivity = 100 * (Q_sensitivity - df_) / Q_sensitivity if Q_sensitivity > df_ else 0
    
    # Append results for analysis
    combined_ORs.append(np.exp(combined_logOR_random_sensitivity))
    I2_values.append(I2_sensitivity)
    
    # Output the result for this iteration
    print(f"After excluding {study['Study']}:")
    print(f"Combined OR: {np.exp(combined_logOR_random_sensitivity):.2f} [{lower_CI_combined_random_sensitivity:.2f}; {upper_CI_combined_random_sensitivity:.2f}]")
    print(f"I²: {I2_sensitivity:.2f}%")
    print("-" * 50)

# Compare the final combined OR and I² from all iterations
print("\nSummary of Sensitivity Analysis:")
print(f"Combined ORs after excluding each study: {combined_ORs}")
print(f"I² values after excluding each study: {I2_values}")

Fixed-effect meta-analysis result:
Combined OR: 1.55 [1.55; 1.56]
I²: 99.97%

Random-effects meta-analysis result:
Combined OR: 2.34 [1.84; 2.96]
I²: 99.97%

Sensitivity Analysis:
After excluding Reece (2007):
Combined OR: 2.37 [1.85; 3.03]
I²: 99.97%
--------------------------------------------------
After excluding Rooban et al. (2008):
Combined OR: 2.40 [1.87; 3.07]
I²: 99.97%
--------------------------------------------------
After excluding Morio et al. (2008):
Combined OR: 2.17 [1.70; 2.77]
I²: 99.97%
--------------------------------------------------
After excluding Gupta et al. (2012):
Combined OR: 2.38 [1.86; 3.05]
I²: 99.97%
--------------------------------------------------
After excluding Nives Protrka et al. (2013):
Combined OR: 1.90 [1.55; 2.33]
I²: 99.95%
--------------------------------------------------
After excluding Rommel et al. (2016):
Combined OR: 2.19 [1.72; 2.78]
I²: 99.97%
--------------------------------------------------
After excluding Shetty et al. (2016):

In [2]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("/Users/rishav/Downloads/archive/train_dataset.csv")
# Create a contingency table for dental caries and smoking
contingency = pd.crosstab(df["dental caries"], df["smoking"])

# Print the table
print("Contingency Table:")
print(contingency)
import numpy as np
from scipy.stats import fisher_exact

# Extract values from the contingency table
a = contingency.loc[1, 1] if 1 in contingency.columns else 0  # Smokers with caries
b = contingency.loc[1, 0] if 0 in contingency.columns else 0  # Non-smokers with caries
c = contingency.loc[0, 1] if 1 in contingency.columns else 0  # Smokers without caries
d = contingency.loc[0, 0] if 0 in contingency.columns else 0  # Non-smokers without caries

# Calculate Odds Ratio
odds_ratio = (a * d) / (b * c) if b * c > 0 else np.inf

# Calculate the standard error of log(OR)
if all(x > 0 for x in [a, b, c, d]):  # Check to avoid division by zero
    se_log_or = np.sqrt(1/a + 1/b + 1/c + 1/d)
    log_or = np.log(odds_ratio)
    ci_lower = np.exp(log_or - 1.96 * se_log_or)
    ci_upper = np.exp(log_or + 1.96 * se_log_or)
else:
    ci_lower, ci_upper = np.nan, np.nan

# Fisher's Exact Test for p-value
_, p_value = fisher_exact([[a, b], [c, d]])

# Print the results
print(f"Odds Ratio (OR): {odds_ratio:.2f}")
print(f"95% Confidence Interval: [{ci_lower:.2f}, {ci_upper:.2f}]")
print(f"P-value: {p_value:.3f}")

Contingency Table:
smoking            0      1
dental caries              
0              20207  10418
1               4459   3900
Odds Ratio (OR): 1.70
95% Confidence Interval: [1.62, 1.78]
P-value: 0.000


In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("/Users/rishav/Downloads/archive/train_dataset.csv")
# Create a contingency table for dental caries and smoking
contingency = pd.crosstab(df["dental caries"], df["smoking"])

# Print the table
print("Contingency Table:")
print(contingency)
import numpy as np
from scipy.stats import fisher_exact

# Extract values from the contingency table
a = contingency.loc[1, 1] if 1 in contingency.columns else 0  # Smokers with caries
b = contingency.loc[1, 0] if 0 in contingency.columns else 0  # Non-smokers with caries
c = contingency.loc[0, 1] if 1 in contingency.columns else 0  # Smokers without caries
d = contingency.loc[0, 0] if 0 in contingency.columns else 0  # Non-smokers without caries

# Calculate Odds Ratio
odds_ratio = (a * d) / (b * c) if b * c > 0 else np.inf

# Calculate the standard error of log(OR)
if all(x > 0 for x in [a, b, c, d]):  # Check to avoid division by zero
    se_log_or = np.sqrt(1/a + 1/b + 1/c + 1/d)
    log_or = np.log(odds_ratio)
    ci_lower = np.exp(log_or - 1.96 * se_log_or)
    ci_upper = np.exp(log_or + 1.96 * se_log_or)
else:
    ci_lower, ci_upper = np.nan, np.nan

# Fisher's Exact Test for p-value
_, p_value = fisher_exact([[a, b], [c, d]])

# Print the results
print(f"Odds Ratio (OR): {odds_ratio:.2f}")
print(f"95% Confidence Interval: [{ci_lower:.2f}, {ci_upper:.2f}]")
print(f"P-value: {p_value:.3f}")

Contingency Table:
smoking            0      1
dental caries              
0              20207  10418
1               4459   3900
Odds Ratio (OR): 1.70
95% Confidence Interval: [1.62, 1.78]
P-value: 0.000
